此份代码用于SIT小组成员调试生成数据集，主要方法已调试好，可方便地在本地运行改进。

In [25]:
#引入所有需要的模块，如有未安装的库建议pip install
# PyAutoGUI是一个纯Python的GUI自动化工具，其目的是可以用程序自动控制鼠标和键盘操作，多平台支持（Windows，OS X，Linux）。
#cv2即opencv2，用于实现电脑摄像头的调用等操作
#PIL 为Python 常用图像处理模块
import pyautogui as pag
import time
import requests
import os
import cv2
import urllib3,base64
import json
from PIL import Image
from urllib.parse import urlencode

### Part 1 收集脸部照片
通过openCV调用摄像头，在按下空格键时保存脸部照片，命名为鼠标所在位置的元组形式。

In [26]:
#将此处改为初始数据集的保存路径
save_path = 'D:/SITdata/'
#获取屏幕大小
screenWidth,screenHeight = pag.size()
#计算获得屏幕原点位置，定为屏幕中心点
origin = (screenWidth/2,screenHeight/2)

cap=cv2.VideoCapture(0) #调用摄像头，0为电脑自带摄像头，1为外部摄像头
while(1):
    ret,frame = cap.read()
    k=cv2.waitKey(1)
    if k==27: #Esc键退出
        break
    elif k==32:#空格键保存图片
        #获取当前鼠标绝对位置
        currMouseX,currMouseY = pag.position() 
        #将鼠标绝对位置转化为相对于原点的相对坐标，并进行归一化处理
        #对归一化处理有疑惑可参考：https://www.jianshu.com/p/95a8f035c86c
        curx=(currMouseX-origin[0])/origin[0]
        cury=(origin[1]-currMouseY)/origin[1]
  
        #curx=round(curx,3)
        #cury=round(cury,3)
        
        curxx=format(curx, '.2f')
        curyy=format(cury, '.2f')
        
        currMouse = (curx,cury)

        
        cv2.imwrite(save_path+curxx+' '+curyy+' .jpg',frame) #修改了一下，方便一些
        print(curxx+','+curyy) #方便调试，若成功保存则会立刻在控制台输出
    cv2.imshow("capture", frame)
cap.release()
cv2.destroyAllWindows()

0.25,-0.33
0.66,0.29
0.13,1.00
-0.88,0.49
-0.69,-0.20
-0.98,-0.77
-0.23,-0.75
0.22,-1.00
0.47,-0.64
1.00,-0.81


### Part 2 裁剪获取眼部图片
通过BaiduAI的人脸识别功能获取脸部特征，并根据所需取的脸部特征点，应用Pillow的crop方法进行裁剪，相关参考文档如下：
1. BaiduAI 人脸识别技术文档：https://ai.baidu.com/ai-doc/FACE/yk37c1u4t
2. Pillow 技术文档：https://pillow.readthedocs.io/en/stable/reference/Image.html

In [27]:
#BaiduAI 发送请求的必备信息，可自行创建应用并修改
access_token = '24.49e2a677615b94c0937fea336ea982b6.2592000.1604821923.282335-22192901'

In [28]:
#BaiduAI 人脸识别方法调用，参数为文件路径，返回BaiduAI平台的反馈
def BaiduMethod(filepath):
    file = open(filepath,'rb')
    img = open(filepath,'rb')
    #参数images：图像base64编码 分别base64编码后的2张图片数据，需urlencode，半角逗号分隔，单次请求最大不超过20M
    img1 = base64.b64encode(img.read())
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"+ "?access_token=" + access_token
    params = {'image':str(img1,'utf-8'),'image_type':'BASE64','face_field':'landmark'}
    headers = {'content-type': 'application/json'}
    return requests.post(request_url, data=params, headers=headers)

以单个文件为例，方便数据集收集方法的调试。

In [29]:
#此处改为所想读取的单个图片
filepath = 'D:/SITdata/(0.13,1.00).jpg'
#考虑到百度AI平台的对并发量的限制，此处推荐分开处理，即获取response后不再重复对同一张照片进行处理
response = BaiduMethod(filepath)
if response:
    print (response.json())

{'error_code': 0, 'error_msg': 'SUCCESS', 'log_id': 5505550555059, 'timestamp': 1602249566, 'cached': 0, 'result': {'face_num': 1, 'face_list': [{'face_token': 'f4d87cac35c2065b81ca1a698aa12f06', 'location': {'left': 215.02, 'top': 222.39, 'width': 178, 'height': 181, 'rotation': -5}, 'face_probability': 1, 'angle': {'yaw': 0.9, 'pitch': -3.81, 'roll': -7.67}, 'landmark': [{'x': 268.19, 'y': 247.09}, {'x': 347.1, 'y': 239.04}, {'x': 311.09, 'y': 279.16}, {'x': 315.54, 'y': 332.76}], 'landmark72': [{'x': 220.32, 'y': 273.21}, {'x': 224.77, 'y': 300.48}, {'x': 231.87, 'y': 327.8}, {'x': 242.03, 'y': 354.46}, {'x': 264.13, 'y': 379.26}, {'x': 293.34, 'y': 392.23}, {'x': 321.32, 'y': 393.36}, {'x': 348.67, 'y': 386.47}, {'x': 374.9, 'y': 368.17}, {'x': 392.07, 'y': 340.12}, {'x': 397.1, 'y': 311.8}, {'x': 398.52, 'y': 283.72}, {'x': 397.55, 'y': 256.04}, {'x': 250.74, 'y': 252.88}, {'x': 258.14, 'y': 245.11}, {'x': 267.19, 'y': 242.01}, {'x': 276.3, 'y': 242.88}, {'x': 285.07, 'y': 249.78}

### 声明
图片切割必须解决如下问题：
眼部切割所得的图片必须是同样大小的矩形，如何实现？
1. 考虑提前定长（如何确保一定包含眼部？不同角度，位置的照片中眼部面积可能不同）
2. 根据设备调整模型？

暂定解决方案：先完整截取眼部图片，随后进行微调尽量删除无效信息，其次通过resize方法调整图片大小至模型所需输入参数。

In [30]:
landmark = response.json()['result']['face_list'][0]['landmark72']
img2 = Image.open(filepath)
#截取方法待调整
box = (landmark[22]['x'],landmark[22]['y'],landmark[22]['x']+150,landmark[22]['y']+50)
img3 = img2.crop(box)
img3.show() #实时显示切割图片方便微调

在调试完眼部图片截取方法后运行该下面模块进行以获取眼部图片数据集。

In [31]:
path = 'D:/SITdata/' #待遍历目录路径
save_path = 'D:/SITdata2/'#保存文件目录
dirs = os.listdir(path)
for file in dirs:
    filepath = os.path.join(path,file)
    response = BaiduMethod(filepath)
    if response and response.json()['error_code']==0:#成功收到且未报错
        landmark = response.json()['result']['face_list'][0]['landmark72']
        img2 = Image.open(filepath)
        #截取方法待调整
        box = (landmark[22]['x'],landmark[22]['y'],landmark[22]['x']+150,landmark[22]['y']+50)
        img3 = img2.crop(box)
        img3.save(save_path+file)